In [29]:
from driver import Navegador
import os
import urllib

In [35]:
from selenium.webdriver.common.by import By


cookie = {
    "name": "li_at",
    "value": os.environ["LINKEDIN_COOKIE"],
    "domain": ".linkedin.com"
}

navegador = Navegador()

navegador.get('https://www.linkedin.com/')
navegador.driver.add_cookie(cookie)
navegador.get('https://www.linkedin.com/')


In [31]:
#Get candidate

def get_candidate_role(role, skills, page):

    role = role.lower()

    if page == '':
        page = 1

    #get by atribute
    skills = skills.split(',')
    search = ' '.join(skills)

    encoded_string = urllib.parse.quote(search.lower())

    url = f"https://www.linkedin.com/search/results/people/?page={page}&keywords={encoded_string}"

    navegador.get(url)



In [32]:
def get_candidates(role, skills):
    from selenium.webdriver.common.by import By
    import time

    navegador.get(f'https://www.linkedin.com/search/results/people/?page=1&keywords={urllib.parse.quote(skills.lower())}')
    # Obtendo a lista de candidatos
    candidates_list = navegador.driver.find_elements(By.CLASS_NAME, "SAsytVGxJmleGgzJzezGdUXVNOHMOEAvpoY")

    # Lista para armazenar os dados dos perfis
    perfis = []

    # Obtendo o número da última página
    ultima_pagina = obter_ultima_pagina(navegador.driver)

    print(f"Ultima pagina: {ultima_pagina}")

    # Iterando sobre cada página
    for page in range(1, ultima_pagina + 1):
        
        get_candidate_role(role, skills, page)
        # faz scroll para baixo
        navegador.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.5)
        for candidate in candidates_list:
            try:
                # Tentando obter o link
                try:
                    link = candidate.find_element(By.TAG_NAME, "a").get_attribute("href")
                except:
                    link = "Link não encontrado"

                # Adicionando os dados à lista de perfis
                perfis.append({
                    "link": link,
                    "nome": "",
                    "skills": "",
                    "sobre": "",
                    "experiencia": [{
                        "cargo": "",
                        "empresa": "",
                        "periodo": "",
                        "local": "",
                        "descricao": ""
                    }],
                    "educacao": [{
                        "curso": "",
                        "instituicao": "",
                        "periodo": "",
                        "descricao": ""
                    }],
                    "certificacoes": [{
                        "certificacao": "",
                        "instituicao": "",
                        "periodo": "",
                        "descricao": ""
                    }],
                    "contato": {
                        "email": "",
                        "telefone": "",
                        "linkedin": "",
                        "github": ""
                    }
                })
            except Exception as e:
                print(f"Erro ao processar candidato: {e}")

        # Printa o numero da pagina
        print(f"Pagina: {page} de {ultima_pagina}")
        # Fala o numero de perfis
        print(f"Perfis encontrados: {len(perfis)}")
        time.sleep(0.5)
        navegador.click("XPATH", "//button[span[text()='Avançar']]")
    # Exibindo os perfis encontrados
    return perfis


def obter_ultima_pagina(driver):
    from selenium.webdriver.common.by import By

    """
    Função para identificar o número da última página em uma lista de paginação.
    
    :param driver: Instância do WebDriver do Selenium.
    :return: Número da última página como inteiro.
    """
    try:
        # Localizar todos os itens de paginação
        paginas = driver.find_elements(By.CSS_SELECTOR, "ul.artdeco-pagination__pages li")
        
        # Iterar pelas páginas para encontrar o maior número
        ultima_pagina = 0
        for pagina in paginas:
            try:
                # Obter o texto dentro do elemento <span>
                texto = pagina.find_element(By.TAG_NAME, "span").text
                # Verificar se o texto é um número
                if texto.isdigit():
                    numero = int(texto)
                    ultima_pagina = max(ultima_pagina, numero)
            except Exception as e:
                # Ignorar casos onde o elemento não possui um número
                pass
        
        return ultima_pagina
    except Exception as e:
        print(f"Erro ao obter a última página: {e}")
        return None




In [33]:
role = 'Desenvolvedor Full Stack'
skills = 'Python, Django, Flask, FastAPI, SQL, NoSQL, Docker, Kubernetes, Git, GitHub, Linux, AWS, Azure, CI/CD, DevOps, Agile, Scrum, Jira, Kanban'

perfis = get_candidates(role, skills)
len(perfis)
for perfil in perfis:
    print(perfil)

Ultima pagina: 0


In [34]:
for perfil in perfis:
    print(perfil)